In [2]:
import os
import pandas as pd
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline
import logging
from tqdm import tqdm
import evaluate
import librosa
import numpy as np
import re

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# GPU 사용 가능 여부 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Using device: {device}")
if device == "cuda":
    logger.info(f"GPU: {torch.cuda.get_device_name(0)}")
    logger.info(f"CUDA Version: {torch.version.cuda}")

# 모델 경로 설정
current_dir = os.getcwd()
original_model = "openai/whisper-small"
fine_tuned_model = "urewui/ktf"

# 평가 데이터 로드
valid_csv = "filtered_data_val.csv"
valid_dir = "valid"

# CER 메트릭 로드
cer_metric = evaluate.load("cer")

def clean_text(text: str) -> str:
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'[",\'\"]', '', text)
    text = re.sub(r'[^\w\s.!?ㄱ-ㅎㅏ-ㅣ가-힣]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def load_audio(audio_path):
    """오디오 파일을 로드하고 전처리합니다."""
    try:
        logger.info(f"오디오 파일 로드 시도: {audio_path}")
        # librosa를 사용하여 오디오 로드
        audio, sr = librosa.load(audio_path, sr=16000)
        logger.info(f"오디오 로드 성공 - 샘플 수: {len(audio)}, 샘플링 레이트: {sr}")
        return audio
    except Exception as e:
        logger.error(f"오디오 파일 로드 중 오류 발생: {e}")
        raise

def load_original_model():
    """원본 모델을 로드합니다."""
    logger.info("원본 모델 로드 중...")
    original_processor = WhisperProcessor.from_pretrained(
        original_model,
        language="korean",
        task="transcribe"
    )
    original_model_pipe = pipeline(
        "automatic-speech-recognition",
        model=original_model,
        device=0 if device == "cuda" else -1,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        chunk_length_s=30,  # 청크 길이 설정
        stride_length_s=5,  # 스트라이드 길이 설정
        batch_size=16,  # 배치 크기 설정
        generate_kwargs={"language": "korean", "task": "transcribe"}  # 생성 시 한국어 설정
    )
    return original_model_pipe

def load_fine_tuned_model():
    """파인튜닝된 모델을 로드합니다."""
    logger.info("파인튜닝된 모델 로드 중...")
    fine_tuned_processor = WhisperProcessor.from_pretrained(fine_tuned_model)
    fine_tuned_model_pipe = pipeline(
        "automatic-speech-recognition",
        model=fine_tuned_model,
        device=0 if device == "cuda" else -1,
        torch_dtype=torch.float32 if device == "cuda" else torch.float32,
        chunk_length_s=30,  # 청크 길이 설정
        stride_length_s=5,  # 스트라이드 길이 설정
        batch_size=16,  # 배치 크기 설정
        generate_kwargs={"language": "korean", "task": "transcribe"}  # 생성 시 한국어 설정
    )
    return fine_tuned_model_pipe

def evaluate_models():
    """두 모델의 성능을 평가합니다."""
    logger.info("모델 평가 시작...")
    
    # 평가 데이터 로드
    if not os.path.exists(valid_csv):
        raise ValueError(f"평가 데이터 파일을 찾을 수 없습니다: {valid_csv}")
        
    valid_df = pd.read_csv(valid_csv)
    logger.info(f"평가 데이터 수: {len(valid_df)}")
    logger.info(f"CSV 컬럼명: {valid_df.columns.tolist()}")
    
    # 컬럼명 매핑 (실제 CSV 파일의 컬럼명에 따라 수정 필요)
    column_mapping = {
        'fileName': 'audio_path',  # 오디오 파일 경로 컬럼
        'Reading': 'clean_ground_truth'  # 정답 텍스트 컬럼
    }
    
    # 컬럼명 변경
    valid_df = valid_df.rename(columns=column_mapping)
    
    # 정답 텍스트 전처리
    valid_df['clean_ground_truth'] = valid_df['clean_ground_truth'].apply(clean_text)
    
    # 결과 저장용 리스트
    fine_tuned_results = []
    original_results = []
    processed_count = 0
    error_count = 0
    
    # 파인튜닝된 모델로 먼저 예측
    logger.info("파인튜닝된 모델 로드 및 예측 시작...")
    fine_tuned_pipe = load_fine_tuned_model()
    fine_tuned_predictions = {}
    
    for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df), desc="파인튜닝된 모델 예측 중"):
        audio_path = os.path.join(valid_dir, row['audio_path'])
        logger.info(f"\n처리 중인 파일 {idx + 1}/{len(valid_df)}: {audio_path}")
        
        if not os.path.exists(audio_path):
            logger.warning(f"오디오 파일을 찾을 수 없습니다: {audio_path}")
            error_count += 1
            continue
        
        try:
            audio = load_audio(audio_path)
            fine_tuned_pred = fine_tuned_pipe(audio)['text']
            fine_tuned_predictions[audio_path] = fine_tuned_pred
            processed_count += 1
            
        except Exception as e:
            logger.error(f"파일 처리 중 오류 발생: {audio_path}, 오류: {e}")
            error_count += 1
            continue
    
    # 파인튜닝된 모델 메모리 해제
    del fine_tuned_pipe
    torch.cuda.empty_cache() if device == "cuda" else None
    
    # 원본 모델로 예측
    logger.info("원본 모델 로드 및 예측 시작...")
    original_pipe = load_original_model()
    
    for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df), desc="원본 모델 예측 중"):
        audio_path = os.path.join(valid_dir, row['audio_path'])
        
        if audio_path not in fine_tuned_predictions:
            continue
            
        try:
            audio = load_audio(audio_path)
            original_pred = original_pipe(audio)['text']
            
            # CER 계산
            fine_tuned_cer = cer_metric.compute(
                predictions=[fine_tuned_predictions[audio_path]], 
                references=[row['clean_ground_truth']]
            )
            original_cer = cer_metric.compute(
                predictions=[original_pred], 
                references=[row['clean_ground_truth']]
            )
            
            # 결과 저장
            fine_tuned_result = {
                'audio_path': row['audio_path'],
                'prediction': fine_tuned_predictions[audio_path],
                'ground_truth': row['clean_ground_truth'],
                'cer': fine_tuned_cer
            }
            fine_tuned_results.append(fine_tuned_result)
            
            original_result = {
                'audio_path': row['audio_path'],
                'prediction': original_pred,
                'ground_truth': row['clean_ground_truth'],
                'cer': original_cer
            }
            original_results.append(original_result)
            
        except Exception as e:
            logger.error(f"파일 처리 중 오류 발생: {audio_path}, 오류: {e}")
            continue
    
    # 원본 모델 메모리 해제
    del original_pipe
    torch.cuda.empty_cache() if device == "cuda" else None
    
    # 결과 처리
    if not fine_tuned_results or not original_results:
        logger.error("처리된 결과가 없습니다. 모든 파일 처리에 실패했습니다.")
        return
    
    # 결과를 DataFrame으로 변환
    fine_tuned_df = pd.DataFrame(fine_tuned_results)
    original_df = pd.DataFrame(original_results)
    
    # 전체 평균 CER 계산
    avg_fine_tuned_cer = fine_tuned_df['cer'].mean()
    avg_original_cer = original_df['cer'].mean()
    
    logger.info(f"처리된 총 샘플 수: {len(fine_tuned_results)}")
    logger.info(f"성공: {processed_count}, 실패: {error_count}")
    logger.info(f"평균 파인튜닝된 모델 CER: {avg_fine_tuned_cer:.4f}")
    logger.info(f"평균 원본 모델 CER: {avg_original_cer:.4f}")
    
    # 결과 저장
    fine_tuned_df.to_csv('fine_tuned_model_results.csv', index=False)
    original_df.to_csv('original_model_results.csv', index=False)
    logger.info("평가 결과가 'fine_tuned_model_results.csv'와 'original_model_results.csv'에 저장되었습니다.")
    
    # 비교 결과 저장
    comparison_df = pd.DataFrame({
        'audio_path': fine_tuned_df['audio_path'],
        'fine_tuned_prediction': fine_tuned_df['prediction'],
        'original_prediction': original_df['prediction'],
        'ground_truth': fine_tuned_df['clean_ground_truth'],
        'fine_tuned_cer': fine_tuned_df['cer'],
        'original_cer': original_df['cer']
    })
    comparison_df.to_csv('model_comparison_results.csv', index=False)
    logger.info("모델 비교 결과가 'model_comparison_results.csv'에 저장되었습니다.")

if __name__ == "__main__":
    evaluate_models() 

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-16 13:56:57.335670: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-16 13:56:57.772092: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-16 13:56:58.937966: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:_

In [3]:
import pandas as pd
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def analyze_improvements():
    """CER이 개선된 케이스를 분석하고 저장합니다."""
    logger.info("CER 개선 케이스 분석 시작...")
    
    # 비교 결과 파일 로드
    try:
        df = pd.read_csv('model_comparison_results.csv')
        logger.info(f"총 {len(df)}개의 샘플 로드됨")
    except FileNotFoundError:
        logger.error("model_comparison_results.csv 파일을 찾을 수 없습니다.")
        return
    
    # CER 개선도 계산 (원본 CER - 파인튜닝 CER)
    df['cer_improvement'] = df['original_cer'] - df['fine_tuned_cer']
    
    # CER이 개선된 케이스만 필터링 (개선도 > 0)
    improved_cases = df[df['cer_improvement'] > 0].copy()
    
    # 개선도 기준으로 내림차순 정렬
    improved_cases = improved_cases.sort_values('cer_improvement', ascending=False)
    
    # 결과 저장
    output_file = 'cer_improved_cases.csv'
    improved_cases.to_csv(output_file, index=False)
    
    # 통계 정보 출력
    total_cases = len(df)
    improved_count = len(improved_cases)
    improvement_rate = (improved_count / total_cases) * 100
    
    logger.info(f"총 샘플 수: {total_cases}")
    logger.info(f"개선된 케이스 수: {improved_count}")
    logger.info(f"개선률: {improvement_rate:.2f}%")
    
    if improved_count > 0:
        avg_improvement = improved_cases['cer_improvement'].mean()
        max_improvement = improved_cases['cer_improvement'].max()
        logger.info(f"평균 개선도: {avg_improvement:.4f}")
        logger.info(f"최대 개선도: {max_improvement:.4f}")
        
        # 상위 5개 개선 케이스 출력
        logger.info("\n상위 5개 개선 케이스:")
        for idx, row in improved_cases.head().iterrows():
            logger.info(f"\n파일: {row['audio_path']}")
            logger.info(f"원본 CER: {row['original_cer']:.4f}")
            logger.info(f"파인튜닝 CER: {row['fine_tuned_cer']:.4f}")
            logger.info(f"개선도: {row['cer_improvement']:.4f}")
            logger.info(f"원본 예측: {row['original_prediction']}")
            logger.info(f"파인튜닝 예측: {row['fine_tuned_prediction']}")
            logger.info(f"정답: {row['ground_truth']}")
    
    logger.info(f"\n개선된 케이스가 {output_file}에 저장되었습니다.")

if __name__ == "__main__":
    analyze_improvements() 

INFO:__main__:CER 개선 케이스 분석 시작...
INFO:__main__:총 576개의 샘플 로드됨
INFO:__main__:총 샘플 수: 576
INFO:__main__:개선된 케이스 수: 545
INFO:__main__:개선률: 94.62%
INFO:__main__:평균 개선도: 0.1214
INFO:__main__:최대 개선도: 0.4259
INFO:__main__:
상위 5개 개선 케이스:
INFO:__main__:
파일: EN16RC131_EN0196_20211031.wav
INFO:__main__:원본 CER: 0.4259
INFO:__main__:파인튜닝 CER: 0.0000
INFO:__main__:개선도: 0.4259
INFO:__main__:원본 예측:  KTX는 2시간반 거리고요. 무근화호는 5시간 거리고, 세마호는 4시간 정도 거리입니다.
INFO:__main__:파인튜닝 예측: 케이티엑스는 두 시간 반 걸리고요 무궁화호는 다섯 시간 걸리고 새마을호는 네 시간 정도 걸립니다.
INFO:__main__:정답: 케이티엑스는 두 시간 반 걸리고요 무궁화호는 다섯 시간 걸리고 새마을호는 네 시간 정도 걸립니다.
INFO:__main__:
파일: EN11RC003_EN0194_20211025.wav
INFO:__main__:원본 CER: 0.3966
INFO:__main__:파인튜닝 CER: 0.0000
INFO:__main__:개선도: 0.3966
INFO:__main__:원본 예측:  용화관에 오기 전에 드려온 고의라면 여기 오기 전에 편에 좀 해드려서 목을 라면과 지수를 하을 산 건 기억나는데 나는데?
INFO:__main__:파인튜닝 예측: 영화관에 오기 전에 들른 곳이라면 여기 오기 전에 편의점에 들러서 먹을 라면과 칫솔을 산 건 기억나는데.
INFO:__main__:정답: 영화관에 오기 전에 들른 곳이라면 여기 오기 전에 편의점에 들러서 먹을 라면과 칫솔을 산 건 기억나는데.
INFO:__main__:
파일: EN16RC13